In [2]:
import pandas as pd
from datasets import Dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
import torch
import os

# Ensure GPU is visible
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Load your dataset from Excel
data = pd.read_excel(r'C:\Users\admin\Desktop\traini\newsdata.xlsx')

# Preprocess function
def preprocess_text(text):
    return text.lower().replace('\n', ' ').replace('\r', '')

# Apply preprocessing
data['text'] = data['text'].apply(preprocess_text)

# Remove classes with fewer than 2 samples
min_samples = 2
value_counts = data['target'].value_counts()
data = data[data['target'].isin(value_counts[value_counts >= min_samples].index)]

# Encode labels
label_encoder = LabelEncoder()
data['target'] = label_encoder.fit_transform(data['target'])
num_labels = len(label_encoder.classes_)

In [11]:

# Split the dataset
train_data, test_data = train_test_split(data, test_size=0.2, stratify=data['target'], random_state=42)

# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_data.reset_index(drop=True))
test_dataset = Dataset.from_pandas(test_data.reset_index(drop=True))

# Load tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Tokenize the texts
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

# Rename the target column to labels
train_dataset = train_dataset.rename_column("target", "labels")
test_dataset = test_dataset.rename_column("target", "labels")

# Set format for PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
print(train_data)
print(train_dataset)


Map: 100%|██████████| 35/35 [00:00<00:00, 2697.60 examples/s]

                                                  text  target
92   with ukraine losing ground, allies debate how ...       6
83   hedge flow hedge funds buy chinese stocks for ...      17
136  japan's factory activity expands for first tim...       5
31   european commission official sees $100 bln in ...       8
96   colorado the first state to move forward with ...      16
..                                                 ...     ...
134  steady, widening service price rises may spur ...      16
156  meta’s new ai council is composed entirely of ...      13
93   weak yuan boosts the appeal of hong kong stock...       2
29   shell sees emerging asian markets taking more ...       6
19   explainer: what new caledonia riots mean for t...       6

[138 rows x 2 columns]
Dataset({
    features: ['text', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 138
})


In [12]:

# Load the model
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=num_labels)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./bert',
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=50,
    weight_decay=0.01,
    no_cuda=not torch.cuda.is_available(),  # Use CUDA if available
    report_to=[]  
)

# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate()
print(f"Evaluation results: {results}")

# Save the model
trainer.save_model('./bert')

# Optionally, save the tokenizer as well
tokenizer.save_pretrained('./bert')

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\admin\Desktop\traini\autotrain-env\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
c:\Users\admin\Desktop\traini\autotrain-env\Lib\site-packages\transformers\training_args.py:1489: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


                                                

                                       
  6%|▌         | 31/540 [1:02:22<04:15,  1.99it/s]



{'eval_loss': 2.666346549987793, 'eval_runtime': 0.2214, 'eval_samples_per_second': 158.053, 'eval_steps_per_second': 22.579, 'epoch': 1.0}




                                                  

                                       
  6%|▌         | 31/540 [1:02:30<04:15,  1.99it/s]



{'eval_loss': 2.4905195236206055, 'eval_runtime': 0.3057, 'eval_samples_per_second': 114.485, 'eval_steps_per_second': 16.355, 'epoch': 2.0}




                                                  

                                       
  6%|▌         | 31/540 [1:02:39<04:15,  1.99it/s]



...




                                                  

                                       
  6%|▌         | 31/540 [1:05:54<04:15,  1.99it/s]



{'eval_loss': 1.7490975856781006, 'eval_runtime': 0.5786, 'eval_samples_per_second': 60.49, 'eval_steps_per_second': 8.641, 'epoch': 23.0}





                                                  

                                       
  6%|▌         | 31/540 [1:06:07<04:15,  1.99it/s]



{'eval_loss': 1.7966868877410889, 'eval_runtime': 0.4266, 'eval_samples_per_second': 82.036, 'eval_steps_per_second': 11.719, 'epoch': 24.0}


KeyboardInterrupt: 

In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import numpy as np
# Load the saved model and tokenizer
model_path = './bert'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Function to classify new titles
def classify_titles(titles):
    results = []
    for title in titles:
        # Tokenize the title
        inputs = tokenizer(title, return_tensors="pt", truncation=True, padding=True)
        
        # Get model predictions
        with torch.no_grad():
            outputs = model(**inputs)
        
        # Get the predicted class and confidence score
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=1).numpy()[0]
        predicted_class_id = np.argmax(probabilities)
        confidence_score = probabilities[predicted_class_id]
        
        # Get the corresponding category
        predicted_category = label_encoder.inverse_transform([predicted_class_id])[0]
        
        # Debugging: print logits, predicted class ID, and confidence score
        print(f"Title: '{title}'")
        print(f"Predicted class ID: {predicted_class_id}")
        print(f"Confidence score: {confidence_score:.4f}")
        
        results.append((title, predicted_category, confidence_score))
    
    return results


# Example titles to classify
titles_to_classify = ["Turkish Rates Likely Already at Peak Even If Inflation Isn’t Yet"]
classified_titles = classify_titles(titles_to_classify)

# Print the results
for title, category, confidence in classified_titles:
    print(f"Title: '{title}' is classified as '{category}'" )


Title: 'Turkish Rates Likely Already at Peak Even If Inflation Isn’t Yet'
Predicted class ID: 5
Confidence score: 0.5957
Title: 'Turkish Rates Likely Already at Peak Even If Inflation Isn’t Yet' is classified as 'Industry & Economic Updates'


In [10]:
import pandas as pd
from datasets import Dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np
import os

# Ensure GPU is visible
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Load your dataset from Excel
data = pd.read_excel(r'C:\Users\admin\Desktop\traini\newsdata.xlsx')

# Preprocess function
def preprocess_text(text):
    return text.lower().replace('\n', ' ').replace('\r', '')

# Apply preprocessing
data['text'] = data['text'].apply(preprocess_text)

# Remove classes with fewer than 2 samples
min_samples = 2
value_counts = data['target'].value_counts()
data = data[data['target'].isin(value_counts[value_counts >= min_samples].index)]

# Encode labels
label_encoder = LabelEncoder()
data['target'] = label_encoder.fit_transform(data['target'])
num_labels = len(label_encoder.classes_)

print(f"Number of classes after filtering: {num_labels}")
print(data['target'].value_counts())

# Split the dataset
train_data, test_data = train_test_split(data, test_size=0.2, stratify=data['target'], random_state=42)

# Tokenizer
model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)

# Tokenize function
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)

# Convert to Hugging Face Dataset
test_dataset = Dataset.from_pandas(test_data.reset_index(drop=True))

# Tokenize the test dataset
test_dataset = test_dataset.map(tokenize, batched=True)

# Rename the target column to labels
test_dataset = test_dataset.rename_column("target", "labels")

# Set format for PyTorch
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Load the saved model and tokenizer
model_path = './results_fold_0'  # Replace with the correct path to your saved model
tokenizer = AutoTokenizer.from_pretrained(r"C:\Users\admin\Desktop\traini\bert")
model = AutoModelForSequenceClassification.from_pretrained(r"C:\Users\admin\Desktop\traini\bert")

# Function to classify new titles and calculate accuracy
def classify_and_evaluate(test_dataset):
    all_predictions = []
    all_labels = []

    for i in range(len(test_dataset)):
        inputs = {
            'input_ids': test_dataset[i]['input_ids'].unsqueeze(0),
            'attention_mask': test_dataset[i]['attention_mask'].unsqueeze(0)
        }
        labels = test_dataset[i]['labels'].unsqueeze(0)

        with torch.no_grad():
            outputs = model(**inputs)
        
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=1).numpy()[0]
        predicted_class_id = np.argmax(probabilities)
        confidence_score = probabilities[predicted_class_id]

        # Append predictions and true labels
        all_predictions.append(predicted_class_id)
        all_labels.append(labels.numpy()[0])

        # Debugging: print logits, predicted class ID, and confidence score
        print(f"Sample {i}")
      
        print(f"Predicted class ID: {predicted_class_id}")
        print(f"Confidence score: {confidence_score:.4f}")
        print(f"True label: {labels.numpy()[0]}")
        print()

    # Calculate accuracy
    accuracy = np.mean(np.array(all_predictions) == np.array(all_labels))
    return accuracy

# Evaluate the model on the test dataset
accuracy = classify_and_evaluate(test_dataset)
print(f"Accuracy on the test dataset: {accuracy:.4f}")


Number of classes after filtering: 19
target
16    34
17    32
10    18
18    13
5     12
6     11
8      9
1      8
2      7
7      6
3      3
0      3
14     3
13     3
11     3
12     2
9      2
4      2
15     2
Name: count, dtype: int64


Map: 100%|██████████| 35/35 [00:00<00:00, 1896.94 examples/s]


Sample 0
Predicted class ID: 5
Confidence score: 0.5957
True label: 16

Sample 1
Predicted class ID: 17
Confidence score: 0.9867
True label: 17

Sample 2
Predicted class ID: 17
Confidence score: 0.9438
True label: 18

Sample 3
Predicted class ID: 10
Confidence score: 0.9556
True label: 10

Sample 4
Predicted class ID: 18
Confidence score: 0.9241
True label: 3

Sample 5
Predicted class ID: 18
Confidence score: 0.7310
True label: 18

Sample 6
Predicted class ID: 17
Confidence score: 0.9838
True label: 17

Sample 7
Predicted class ID: 5
Confidence score: 0.3041
True label: 7

Sample 8
Predicted class ID: 10
Confidence score: 0.5122
True label: 11

Sample 9
Predicted class ID: 5
Confidence score: 0.6100
True label: 5

Sample 10
Predicted class ID: 10
Confidence score: 0.6871
True label: 14

Sample 11
Predicted class ID: 10
Confidence score: 0.6826
True label: 8

Sample 12
Predicted class ID: 10
Confidence score: 0.8716
True label: 8

Sample 13
Predicted class ID: 10
Confidence score: 0.479

In [1]:
import pandas as pd
from datasets import Dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer, AutoModelForSequenceClassification
from transformers import EarlyStoppingCallback
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch
import numpy as np
import os

# Ensure GPU is visible
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Load your dataset from Excel
data = pd.read_excel(r'C:\Users\admin\Desktop\traini\newsdata.xlsx')

# Preprocess function
def preprocess_text(text):
    return text.lower().replace('\n', ' ').replace('\r', '')

# Apply preprocessing
data['text'] = data['text'].apply(preprocess_text)

# Remove classes with fewer than 2 samples
min_samples = 2
value_counts = data['target'].value_counts()
data = data[data['target'].isin(value_counts[value_counts >= min_samples].index)]

# Encode labels
label_encoder = LabelEncoder()
data['target'] = label_encoder.fit_transform(data['target'])
num_labels = len(label_encoder.classes_)

print(f"Number of classes after filtering: {num_labels}")
print(data['target'].value_counts())

# Tokenizer
model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)

# Tokenize function
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)

# Initialize cross-validation
kf = StratifiedKFold(n_splits=5)

# Cross-validation training and evaluation
accuracies = []
precisions = []
recalls = []
f1_scores = []

for fold, (train_index, val_index) in enumerate(kf.split(data, data['target'])):
    print(f"Training fold {fold + 1}")
    
    train_data = data.iloc[train_index].reset_index(drop=True)
    val_data = data.iloc[val_index].reset_index(drop=True)

    # Convert to Hugging Face Dataset
    train_dataset = Dataset.from_pandas(train_data)
    val_dataset = Dataset.from_pandas(val_data)
    
    # Tokenize the datasets
    train_dataset = train_dataset.map(tokenize, batched=True)
    val_dataset = val_dataset.map(tokenize, batched=True)

    # Rename the target column to labels
    train_dataset = train_dataset.rename_column("target", "labels")
    val_dataset = val_dataset.rename_column("target", "labels")

    # Set format for PyTorch
    train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
    val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

    # Load the model
    model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

    # Define training arguments
    training_args = TrainingArguments(
        output_dir=f'./results_fold_{fold}',
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=3e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=10,
        weight_decay=0.01,
        use_cpu=not torch.cuda.is_available(),
        report_to=[],
        save_total_limit=2,
        load_best_model_at_end=True,
    )

    # Create a Trainer instance
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )

    # Train the model
    trainer.train()

    # Evaluate the model
    results = trainer.evaluate()
    print(f"Evaluation results for fold {fold + 1}: {results}")

    # Save the model and tokenizer
    model.save_pretrained(f'./results_fold_{fold}')
    tokenizer.save_pretrained(f'./results_fold_{fold}')
    
    # Calculate predictions and true labels
    predictions = np.argmax(trainer.predict(val_dataset).predictions, axis=1)
    true_labels = val_dataset['labels'].numpy()
    
    # Calculate metrics
    accuracy = accuracy_score(true_labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='weighted')
    
    # Append metrics for this fold
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)

# Calculate average metrics across all folds
average_accuracy = np.mean(accuracies)
average_precision = np.mean(precisions)
average_recall = np.mean(recalls)
average_f1_score = np.mean(f1_scores)

print(f"Average accuracy: {average_accuracy:.4f}")
print(f"Average precision: {average_precision:.4f}")
print(f"Average recall: {average_recall:.4f}")
print(f"Average F1-score: {average_f1_score:.4f}")

# Determine the best model based on average F1-score (or another metric)
best_fold_index = np.argmax(f1_scores)
print(f"The best model is from fold {best_fold_index + 1}")

# Load the best model for final evaluation on the test set
model_path = f'./results_fold_{best_fold_index}'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Function to classify new titles and calculate accuracy on the test set
def classify_and_evaluate(test_dataset):
    all_predictions = []
    all_labels = []

    for i in range(len(test_dataset)):
        inputs = {
            'input_ids': test_dataset[i]['input_ids'].unsqueeze(0),
            'attention_mask': test_dataset[i]['attention_mask'].unsqueeze(0)
        }
        labels = test_dataset[i]['labels'].unsqueeze(0)

        with torch.no_grad():
            outputs = model(**inputs)
        
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=1).numpy()[0]
        predicted_class_id = np.argmax(probabilities)
        confidence_score = probabilities[predicted_class_id]

        # Append predictions and true labels
        all_predictions.append(predicted_class_id)
        all_labels.append(labels.numpy()[0])

        # Debugging: print logits, predicted class ID, and confidence score
        print(f"Sample {i}")
        print(f"Logits: {logits}")
        print(f"Predicted class ID: {predicted_class_id}")
        print(f"Confidence score: {confidence_score:.4f}")
        print(f"True label: {labels.numpy()[0]}")
        print()

    # Calculate accuracy
    accuracy = np.mean(np.array(all_predictions) == np.array(all_labels))
    return accuracy

# Convert test data to Hugging Face Dataset and tokenize
test_data = data.iloc[val_index].reset_index(drop=True)
test_dataset = Dataset.from_pandas(test_data)
test_dataset = test_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.rename_column("target", "labels")
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Evaluate the model on the test dataset
accuracy = classify_and_evaluate(test_dataset)
print(f"Accuracy on the test dataset: {accuracy:.4f}")




c:\Users\admin\Desktop\traini\autotrain-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Number of classes after filtering: 19
target
16    34
17    32
10    18
18    13
5     12
6     11
8      9
1      8
2      7
7      6
3      3
0      3
14     3
13     3
11     3
12     2
9      2
4      2
15     2
Name: count, dtype: int64


c:\Users\admin\Desktop\traini\autotrain-env\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


Training fold 1


Map: 100%|██████████| 35/35 [00:00<00:00, 2706.55 examples/s]
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\admin\Desktop\traini\autotrain-env\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
                                                
 10%|█         | 18/180 [00:08<01:17,  2.09it/s]

...

Evaluation results for fold 5: {'eval_loss': 2.285047769546509, 'eval_runtime': 0.218, 'eval_samples_per_second': 155.93, 'eval_steps_per_second': 22.931, 'epoch': 10.0}


100%|██████████| 5/5 [00:00<00:00, 29.93it/s]
c:\Users\admin\Desktop\traini\autotrain-env\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Average accuracy: 0.4099
Average precision: 0.3163
Average recall: 0.4099
Average F1-score: 0.3231
The best model is from fold 1


Map: 100%|██████████| 34/34 [00:00<00:00, 5681.30 examples/s]


Sample 0
Logits: tensor([[-0.5147,  0.1452, -0.1827,  0.3754, -0.4275,  0.1474,  0.1920, -0.2378,
         -0.0086, -0.8068,  0.6469, -0.7972, -0.3134, -0.6750, -0.5901, -0.6189,
          1.0171,  0.4323,  0.5441]])
Predicted class ID: 16
Confidence score: 0.1382
True label: 3

Sample 1
Logits: tensor([[-0.2758, -0.1615, -0.1410, -0.4631, -0.6451,  0.5631,  1.4634,  0.5597,
          0.0450, -0.6723,  0.4159, -0.5304, -0.3976, -0.7369, -0.6446, -0.5435,
          1.1181,  0.6323,  0.0744]])
Predicted class ID: 6
Confidence score: 0.1847
True label: 6

Sample 2
Logits: tensor([[-0.1676, -0.1231, -0.1395, -0.3573, -0.5591,  0.5704,  1.4357,  0.4759,
         -0.0123, -0.7218,  0.4429, -0.6238, -0.3513, -0.7556, -0.6621, -0.4805,
          0.9892,  0.5345,  0.0711]])
Predicted class ID: 6
Confidence score: 0.1844
True label: 6

Sample 3
Logits: tensor([[-0.1351, -0.2074, -0.0946, -0.4974, -0.6130,  0.6034,  1.0048,  0.8198,
         -0.0852, -0.6555,  0.2955, -0.3564, -0.3423, -0.5509, -